# [STARTER] Exercise - Output structured Agent responses

In this exercise, you'll learn how to enhance your AI agent to provide structured outputs using Pydantic models. This will help ensure the agent's responses are consistent, validated, and easily usable in downstream applications.

## Challenge

You have an existing Agent class that can:
- Process user messages
- Use tools when needed
- Generate responses

Now you need to enhance it to:
- Define structured output formats using Pydantic
- Parse and validate responses
- Return data in a consistent JSON format



## Setup
First, let's import the necessary libraries:

In [1]:
from typing import List, Any, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

# lib module imports messages, tooling, LLM interaction, and output parsing 
from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM
from lib.parsers import PydanticOutputParser, JsonOutputParser

## Defining Structured Output Models

Let's create a Pydantic model for a meeting summary with action items:


In [2]:
# TODO 1: Create the ActionItem Pydantic model
# Hint: Include fields for task, assignee, and due_date with appropriate annotations and descriptions

class ActionItem(BaseModel):
    """A pydantic model representing an action item extracted from a meeting summary.
    the follwoing fields are excpected:
     - task: A description of the task to be completed.
     - assignee: The person responsible for completing the task.
     - due_date: The deadline for completing the task, in YYYY-MM-DD format.
    """
    task: Annotated[str, Field(..., description="A description of the task to be completed.")]
    assignee: Annotated[str, Field(..., description="The person responsible for completing the task.")]
    due_date: Annotated[str, Field(..., description="The deadline for completing the task, in YYYY-MM-DD format.")]

In [3]:
# TODO 2: Create the MeetingSummary Pydantic model
# Hint: Include fields for title, date, participants, key_points, and action_items

class MeetingSummary(BaseModel):
    """A pydantic model representing a meeting summary.
    The following fields are expected:
     - title: The title of the meeting.
     - date: The date of the meeting, in YYYY-MM-DD format.
     - participants: A list of participants who attended the meeting.
     - key_points: A list of key points discussed during the meeting.
     - action_items: A list of action items extracted from the meeting summary, represented as ActionItem instances.
    """
    title: Annotated[str, Field(..., description="The title of the meeting.")]
    date: Annotated[str, Field(..., description="The date of the meeting, in YYYY-MM-DD format.")]
    participants: Annotated[List[str], Field(..., description="A list of participants who attended the meeting.")]
    key_points: Annotated[List[str], Field(..., description="A list of key points discussed during the meeting.")]
    action_items: Annotated[List[ActionItem], Field(..., description="A list of action items extracted from the meeting summary, represented as ActionItem instances.")]

## Enhanced Agent Class

Now let's create an enhanced version of our Agent class that supports structured outputs:


In [19]:
class StructuredAgent:
    """An AI Agent that provides structured outputs"""
    
    def __init__(
        self,
        role: str = "Meeting Assistant",
        instructions: str = "Help summarize meetings and track action items",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None,
        output_model: BaseModel = None
    ):
        """Initialize the agent with its configuration
        
        Args:
            role: The agent's role/persona
            instructions: Basic instructions for the agent
            model: The LLM model to use
            temperature: Creativity parameter (0.0 = more deterministic)
            tools: List of tools the agent can use
            output_model: Pydantic model for structured output
        """
        # TODO 3: Initialize the agent
        # Hint:
        # - Store agent settings (role, instructions, output_model, etc.)
        # - Load environment variables
        # - Create an LLM instance with the provided configuration

        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.tools = tools or []
        self.output_model = output_model # Store the provided pydantic output structure (e.g., MeetingSummary) for later use in parsing LLM responses into structured formats.

    def invoke(self, user_message: str) -> dict: # -> dict indicates that this method is expected to return a dictionary
        """Process a user message and return a structured response
        
        Args:
            user_message: The user's input message
            
        Returns:
            A dictionary containing the structured response
        """
        # TODO 4: Implement the invoke method
        # Hint:
        # - Create messages list with SystemMessage
        # - Add UserMessage
        # - Get AI response with structured format if output_model exists
        # - Parse and return the response

        messages = [
            SystemMessage( # create a system message as per the format in the lib.messages module, which provides context and instructions to the LLM about its role and how it should respond to user messages.
                content=( # "(" informs python that its an implicit string concatenation
                    f"You are a helpful AI agent assistant your role is {self.role}."
                    f"Your instructions are {self.instructions}."


                )
            )
        ]
                    # "(" above informs python that its an implicit string concatenation, allowing us to build a longer string across multiple lines for better readability.
                    # note the above joins the below into one string e.g. "...your role is Meeting Assistant.Your instructions are..."

                    # The below introduces a new line between the sentences and passes it to the LLM e.g.
                    # content=f"""You are a helpful AI agent assistant your role is {self.role}.
                    # Your instructions are {self.instructions}."""

        load_dotenv() # Load environment variables from a .env file, which may include API keys or other configuration needed for the LLM to function properly.

        self.llm_chat_model = LLM(model=self.model, temperature=self.temperature, tools=self.tools) # Create an instance of the LLM class with the specified model, temperature, and tools. This instance will be used to interact with the language model and generate responses based on the input messages.

        messages.append(UserMessage(content=user_message)) # Add the user's message to the list of messages that will be sent to the LLM for processing.

        if self.output_model: # If an output model is defined, use the PydanticOutputParser to parse the response into a structured format

            parser = PydanticOutputParser(model_class=self.output_model) # Create a parser configured to validate data is in the
            # specified Pydantic model format (e.g., MeetingSummary).

            ai_message = self.llm_chat_model.invoke(messages, response_format=self.output_model) 
            # Get the AI response, passing the parser's format instructions to ensure the response is structured correctly for parsing. 
            
            structured_response = parser.parse(ai_message)
            return structured_response.model_dump()  # model_dump() converts the pydantic model instance into a dictionary format as per the expected return type of the invoke method.
        else:
            ai_message = self.llm_chat_model.invoke(messages) # If no pydantic output model is defined, simply get the AI response without structured formatting.
            return {"response": ai_message.content}

## Testing the Structured Agent

Let's test our enhanced agent with a meeting summary example:


In [20]:
# Create an agent instance with the MeetingSummary model
meeting_agent = StructuredAgent(
    role="Meeting Assistant",
    instructions="Summarize meetings and track action items in a structured format",
    output_model=MeetingSummary
)

In [21]:
meeting_transcript = """
Project Planning Meeting - March 15, 2024

Attendees: John, Sarah, Mike

Discussion:
- Reviewed Q1 project timeline
- Discussed resource allocation
- Identified potential risks

Next steps:
1. John will update the project plan by next Friday
2. Sarah needs to coordinate with the design team by Wednesday
3. Mike will prepare the risk assessment document by end of month
"""

In [ ]:
summary = meeting_agent.invoke(meeting_transcript)
print(json.dumps(summary, indent=2)) # Print the structured meeting summary in a readable JSON format. 
# indent=2 formats the JSON output with indentation for better readability. 
# Specifically indent = 2 means that each level of the JSON structure will be indented by 2 spaces, making it easier to read 
# and understand the hierarchy of the data.

{
  "title": "Project Planning Meeting",
  "date": "2024-03-15",
  "participants": [
    "John",
    "Sarah",
    "Mike"
  ],
  "key_points": [
    "Reviewed Q1 project timeline",
    "Discussed resource allocation",
    "Identified potential risks"
  ],
  "action_items": [
    {
      "task": "Update the project plan",
      "assignee": "John",
      "due_date": "2024-03-22"
    },
    {
      "task": "Coordinate with the design team",
      "assignee": "Sarah",
      "due_date": "2024-03-20"
    },
    {
      "task": "Prepare the risk assessment document",
      "assignee": "Mike",
      "due_date": "2024-03-31"
    }
  ]
}


## Validating the Output

Let's verify that our output matches our Pydantic model structure:


In [ ]:
# Create a MeetingSummary instance from the output
validated_summary = MeetingSummary(**summary) # The double asterisks (**) are used to unpack the summary dictionary into keyword arguments
# that match the fields of the MeetingSummary model, allowing for validation and structured access to the data.

In [ ]:
# Access structured data
print("Meeting Title:", validated_summary.title) # Access the title field from the validated_summary instance, which is an instance of the MeetingSummary pydantic model.
print("\nParticipants:")
for participant in validated_summary.participants:
    print(f"- {participant}")

print("\nAction Items:")
for item in validated_summary.action_items:
    print(f"- {item.task} (Assigned to: {item.assignee}, Due: {item.due_date})")

Meeting Title: Project Planning Meeting

Participants:
- John
- Sarah
- Mike

Action Items:
- Update the project plan (Assigned to: John, Due: 2024-03-22)
- Coordinate with the design team (Assigned to: Sarah, Due: 2024-03-20)
- Prepare the risk assessment document (Assigned to: Mike, Due: 2024-03-31)
